# Observational study 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotnine
#from networkx import nx
from plotnine import ggplot, aes, geom_bar
from functions import *
import spacy
import time
import datetime

In [2]:
from popularity import trending_history
from financials import *

In [3]:
elon_org_df = pd.read_csv('Data/org-lg-Elon Musk.csv.bz2')

## Matching : 

In [4]:
def compareCompanies(companies, tickers, date, shares, hist):
    # Returns a df with the following columns: ['Date', 'Close', 'Volume', 'MarketCap', 'Popularity']
    df = pd.DataFrame(companies, columns=['Name'])
    df['Date'] = date
    
    marketData = []

    for i in range(len(tickers)):
        marketData.append(obs_info(tickers[i], date, shares[i], hist[i]).values.tolist())
    
    df['Close'] = [i[0] for i in marketData]
    df['Volume'] = [i[1] for i in marketData]
    df['MarketCap'] = [i[2] for i in marketData]
    
    df_trend = trending_history(companies, date)
    df_trend_list = df_trend.values.tolist()[0]
    df['Popularity'] = df_trend_list
    
    return df

def obs_info(ticker, date, sharesOutstanding, comp_hist):
    for i in range(len(comp_hist)):
        if comp_hist.iloc[i].name.date() > datetime.date.fromisoformat(date):
            df = comp_hist.iloc[i]
            break

    df['MarketCap'] = sharesOutstanding*df['Close']
    
    return df

In [5]:
data = pd.DataFrame(columns = ['Name','Date','Close','Volume','MarketCap','Popularity','Elon','compare'])

#Data is our data frame with all the companies with first column 1 or 0 if Elon Musk talks about it.
# after,Name of the company and the other four covariates 
def compare(company_quote, date, companies, tickers, shares, hist): 

    company = compareCompanies(companies, tickers, date, shares, hist)

    company ['Elon']= 0
    company ['compare'] = company_quote
    company.at[4,'Elon'] = 1
    company.at[4,'compare'] = 'None'
    return company
    
#Call this function for all our functions and a set of company that will represent the controled one.

In [10]:
#Apple : 
def create_data(companies_list):
    Company_Apple = companies_list
    quotes_Apple = elon_org_df[elon_org_df['ORG'] == Company_Apple[-1]]
    first_date = quotes_Apple.iloc[0]['date'][0:10]
    ticker_Apple =[ticker_of_company(Company_Apple[0]),
                   ticker_of_company(Company_Apple[1]),
                   ticker_of_company(Company_Apple[2]),
                   ticker_of_company(Company_Apple[3]),
                  ticker_of_company(Company_Apple[4])] 
    shares_Apple =[yf.Ticker(ticker_Apple[0]).info['sharesOutstanding'],
                  yf.Ticker(ticker_Apple[1]).info['sharesOutstanding'],
                  yf.Ticker(ticker_Apple[2]).info['sharesOutstanding'],
                  yf.Ticker(ticker_Apple[3]).info['sharesOutstanding'],
                   yf.Ticker(ticker_Apple[4]).info['sharesOutstanding']]
    hist_Apple =[yf.Ticker(ticker_Apple[0]).history(start=first_date)[['Close', 'Volume']],
                  yf.Ticker(ticker_Apple[1]).history(start=first_date)[['Close', 'Volume']],
                  yf.Ticker(ticker_Apple[2]).history(start=first_date)[['Close', 'Volume']],
                  yf.Ticker(ticker_Apple[3]).history(start=first_date)[['Close', 'Volume']],
                   yf.Ticker(ticker_Apple[4]).history(start=first_date)[['Close', 'Volume']]]

    for i in range (len(quotes_Apple)):
        date = quotes_Apple.iloc[i,:]['date'][0:10]
        before =time.time()
        company = compare('Apple', date, Company_Apple, ticker_Apple, shares_Apple, hist_Apple)
        after =time.time()
        print('fetch time:',str(after-before))
        frame = [data, company]
        data = pd.concat(frame)
    return data

In [ ]:
create_data(['Microsoft','IBM','Samsung','Dell','Apple'] )

In [7]:
data.to_csv('Data/ObservationalDat.csv.bz2' , index=False)

In [8]:
data = pd.read_csv('Data/ObservationalDat.csv.bz2')
data

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare
0,Microsoft,2015-10-09,42.247684,19769066.0,3.171948e+11,30,0,Apple
1,IBM,2015-10-09,114.678062,3227636.0,1.027882e+11,3,0,Apple
2,Samsung,2015-10-09,21274.500000,18948700.0,1.270041e+14,100,0,Apple
3,Dell,2015-10-09,42.173302,92700.0,1.222946e+10,15,0,Apple
4,Apple,2015-10-09,108.199104,30467204.0,1.775158e+12,54,1,None
...,...,...,...,...,...,...,...,...
465,Microsoft,2020-03-12,156.284302,92727446.0,1.173379e+12,36,0,Apple
466,IBM,2020-03-12,98.677086,12502113.0,8.844625e+10,3,0,Apple
467,Samsung,2020-03-12,47106.953125,59462933.0,2.812182e+14,100,0,Apple
468,Dell,2020-03-12,35.950001,4887450.0,1.042482e+10,15,0,Apple


In [9]:
#Tesla : 
Company_Tesla = ['Toyota','BMW','VolksWagen','General Motors']
quotes_Tesla = elon_org_df[elon_org_df['ORG']== 'Tesla']
print ( len(quotes_Tesla))
for i in range (20):
    print (i)
    date = quotes_Tesla.iloc[i,:]['date'][0:10]
    company = compare('Tesla',date,Company_Tesla)
    frame = [data, company]
    data = pd.concat(frame)

2208
0


TypeError: compare() missing 3 required positional arguments: 'tickers', 'shares', and 'hist'

In [ ]:
data.to_csv('Data/ObservationalDat.csv.bz2')

In [ ]:
dates = quotes_Tesla['date'].unique()
print (len(dates))


In [ ]:
for i in range (1000):
    print (i)
    date = quotes_Tesla.iloc[i+20,:]['date'][0:10]
    company = compare('Tesla',date,Company_Tesla)
    frame = [data, company]
    data = pd.concat(frame)

In [ ]:
data


In [ ]:
data.to_csv('Data/ObservationalDat.csv.bz2')

In [ ]:
for i in range (1000):
    print (i)
    date = quotes_Tesla.iloc[i+36,:]['date'][0:10]
    company = compare('Tesla',date,Company_Tesla)
    frame = [data, company]
    data = pd.concat(frame)

In [ ]:
data

In [ ]:
data.to_csv('Data/ObservationalDat.csv.bz2')

In [ ]:
for i in range (1000):
    print (i)
    date = quotes_Tesla.iloc[i+20,:]['date'][0:10]
    company = compare('Tesla',date,Company_Tesla)
    frame = [data, company]
    data = pd.concat(frame)

In [ ]:
#Twitter : 
Company_Twitter = ['Google','Facebook','Snapchat','Pinterest']
quotes_Twitter = elon_org_df[elon_org_df['ORG']== 'Twitter']
print (len(quotes_Twitter))
for i in range (len(quotes_Twitter)):
    print (i)
    date = quotes_Twitter.iloc[i,:]['date'][0:10]
    company = compare('Twitter',date,Company_Twitter)
    frame = [data, company]
    data = pd.concat(frame)

In [ ]:
#Ford : 
Company_Ford = ['Toyota','Fiat','General Motors','Stellantis']
quotes_Ford = elon_org_df[elon_org_df['ORG']=='Ford']
print (len(quotes_Ford))
for i in range (len(quotes_Ford)):
    print (i)
    date = quotes_Ford.iloc[i,:]['date'][0:10]
    company = compare('Ford',date,Company_Ford)
    frame = [data, company]
    data = pd.concat(frame)

In [ ]:
#Paypal :
Company_PayPal = ['Western Union','EuroNet','MoneyGram','Payoneer']
quotes_PayPal = elon_org_df[elon_org_df['ORG']== 'PayPal']
print ( len(quotes_PayPal))
for i in range (len(quotes_PayPal)):
    print (i)
    date = quotes_PayPal.iloc[i,:]['date'][0:10]
    company = compare('PayPal',date,Company_PayPal)
    frame = [data, company]
    data = pd.concat(frame)

In [ ]:
data

In [ ]:
#Propensity score : 
def add_propensityscore (companies, date):#so that if we want to change only the first line don't have to do it
    #With a column of one and zero if elon musk talked about you 
    #company = company_quote.extend(controled)
    data = compareCompanies(company,date)
    #Normalise the features 
    data['Close'] = (data['Close'] - data['Close'].mean())/lalonde_data['Close'].std()
    data['Volume']= (data['Volume']-data['Volume'].mean())/data['Volume'].std()
    data['MarketCap']= (data['MarketCap']-data['MarketCap'].mean())/data['MarketCap'].std()
    data['Popularity']= (data['Popularity']-data['Popularity'].mean())/data['Popularity'].std()
    #Create the model
    mod = smf.logit(formula='elon ~ Close+ Volume + MarketCap + Popularity', data=data)
    res = mod.fit()
    data['Propensity_score'] = res.predict()

In [ ]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [ ]:
def matching(companies):
    treatment_df = data[data['elon'] == 1]
    control_df   = data[data['elon'] == 0]
    # Create an empty undirected graph
    G = nx.Graph()
    # Loop through all the pairs of instances
    for control_id, control_row in control_df.iterrows():
        for treatment_id, treatment_row in treatment_df.iterrows():
            if (controle_row['compare']== treatment_id) and (controle_row['date']== treatment_row['date']) : 
                # Calculate the similarity 
                similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])
                # Add an edge between the two instances weighted by the similarity between them
                G.add_weighted_edges_from([(control_id, treatment_id, similarity)])
    # Generate and return the maximum weight matching on the generated graph
    matching = nx.max_weight_matching(G)
    return matching

In [ ]:
matching = matching(data)
matched = [i[0] for i in list(matching)]+ [i[1]for in list(matching)]
balanced = data.iloc[matched]


## After matching : 


In [ ]:
before =time.time()
quotes_Apple = elon_org_df[elon_org_df['ORG'] == 'Apple']
first_date = quotes_Apple.iloc[0]['date'][0:10]
Company_Apple =['Microsoft','IBM','Samsung','Dell','Apple'] 
ticker_Apple =[ticker_of_company('Microsoft'),
               ticker_of_company('IBM'),
               ticker_of_company('Samsung'),
               ticker_of_company('Dell'),
              ticker_of_company('Apple')] 
shares_Apple =[yf.Ticker(ticker_Apple[0]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[1]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[2]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[3]).info['sharesOutstanding'],
               yf.Ticker(ticker_Apple[4]).info['sharesOutstanding']]
hist_Apple =[yf.Ticker(ticker_Apple[0]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[1]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[2]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[3]).history(start=first_date)[['Close', 'Volume']],
               yf.Ticker(ticker_Apple[4]).history(start=first_date)[['Close', 'Volume']]]
after =time.time()
print('fetch time:',str(after-before))

In [ ]:
hist_Apple[0].iloc[0]

In [ ]:
hist_Apple[0].iloc[0].name

In [ ]:
hist_Apple[0].iloc[0].name.date() > datetime.date.fromisoformat('2015-12-04')

In [ ]:
for i in range(len(hist_Apple[0])):
    if hist_Apple[0].iloc[i].name.date() > datetime.date.fromisoformat('2015-12-12'):
        print(hist_Apple[0].iloc[i])
        break